In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet_v2 import ResNet50V2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224, 224]
CHANNELS=3


In [13]:
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import shutil

filenames1=['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec','healthy']
filenames=['Acne and Rosacea Photos','Atopic Dermatitis Photos','Bullous Disease Photos','Cellulitis Impetigo and other Bacterial Infections','Eczema Photos','Exanthems and Drug Eruptions', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos',  'Light Diseases and Disorders of Pigmentation',  'Lupus and other Connective Tissue diseases', 'Nail Fungus and other Nail Disease',  'Poison Ivy Photos and other Contact Dermatitis',  'Psoriasis pictures Lichen Planus and related diseases',  'Scabies Lyme Disease and other Infestations and Bites',  'Systemic Disease',   'Tinea Ringworm Candidiasis and other Fungal Infections','Urticaria Hives','Vascular Tumors', 'Vasculitis Photos','Warts Molluscum and other Viral Infections']

for filename in filenames:

    src_dir = f'C:/Users/RAHUL NAIR/Documents/all folders/Project folder/HAM dataset/synthetic_files/train/{filename}/'

    # Set the path to the new directory where the selected images will be stored
    dst_dir = f'C:/Users/RAHUL NAIR/Documents/all folders/Project folder/HAM dataset/synthetic_files/train_final/{filename}/'
    dst_dir1 = f'C:/Users/RAHUL NAIR/Documents/all folders/Project folder/HAM dataset/synthetic_files/val/{filename}/'

    # Create the new directory if it doesn't exist
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    if not os.path.exists(dst_dir1):
        os.makedirs(dst_dir1)
    # Get a list of the images in the directory
    images = os.listdir(src_dir)
    len(images)
    
# Pandas dataframe
    df = pd.DataFrame({'image': images})
    train_images, test_images = train_test_split(df, test_size=0.1, random_state=42)
    for image in train_images["image"]:
        src_path = os.path.join(src_dir, image)
        dst_path = os.path.join(dst_dir, image)
        shutil.copy(src_path, dst_path)
    for image in test_images["image"]:
        src_path1 = os.path.join(src_dir, image)
        dst_path1 = os.path.join(dst_dir1, image)
        shutil.copy(src_path1, dst_path1)
# images = os.listdir(src_dir)
# print(images)
# len(images)

In [3]:
train_path = '../HAM dataset/synthetic_files/train_final'
test_path = '../HAM dataset/synthetic_files/test'
val_path = '../HAM dataset/synthetic_files/val'

In [4]:
resnet = ResNet50V2(input_shape=IMAGE_SIZE + [CHANNELS], weights='imagenet', include_top=False)

In [5]:
for layer in resnet.layers:
  layer.trainable = False

In [6]:
folders = glob('../HAM dataset/synthetic_files/train/*')
folders

['../HAM dataset/synthetic_files/train\\Acne and Rosacea Photos',
 '../HAM dataset/synthetic_files/train\\akiec',
 '../HAM dataset/synthetic_files/train\\Atopic Dermatitis Photos',
 '../HAM dataset/synthetic_files/train\\bcc',
 '../HAM dataset/synthetic_files/train\\bkl',
 '../HAM dataset/synthetic_files/train\\Bullous Disease Photos',
 '../HAM dataset/synthetic_files/train\\Cellulitis Impetigo and other Bacterial Infections',
 '../HAM dataset/synthetic_files/train\\df',
 '../HAM dataset/synthetic_files/train\\Eczema Photos',
 '../HAM dataset/synthetic_files/train\\Exanthems and Drug Eruptions',
 '../HAM dataset/synthetic_files/train\\Hair Loss Photos Alopecia and other Hair Diseases',
 '../HAM dataset/synthetic_files/train\\healthy',
 '../HAM dataset/synthetic_files/train\\Herpes HPV and other STDs Photos',
 '../HAM dataset/synthetic_files/train\\Light Diseases and Disorders of Pigmentation',
 '../HAM dataset/synthetic_files/train\\Lupus and other Connective Tissue diseases',
 '../HAM

In [7]:

x = Flatten()(resnet.output)
prediction = Dense(len(folders), activation='softmax')(x)


In [8]:
model = Model(inputs=resnet.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
testing_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

testing_set=testing_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


# '''r=model.fit_generator(training_set,
#                          samples_per_epoch = 8000,
#                          nb_epoch = 5,
#                          validation_data = test_set,
#                          nb_val_samples = 2000)'''


Found 91174 images belonging to 28 classes.
Found 10142 images belonging to 28 classes.
Found 33780 images belonging to 28 classes.


In [12]:
class_names=training_set.class_indices
# class_names=class_names.keys()
class_names

{'Acne and Rosacea Photos': 0,
 'Atopic Dermatitis Photos': 1,
 'Bullous Disease Photos': 2,
 'Cellulitis Impetigo and other Bacterial Infections': 3,
 'Eczema Photos': 4,
 'Exanthems and Drug Eruptions': 5,
 'Hair Loss Photos Alopecia and other Hair Diseases': 6,
 'Herpes HPV and other STDs Photos': 7,
 'Light Diseases and Disorders of Pigmentation': 8,
 'Lupus and other Connective Tissue diseases': 9,
 'Nail Fungus and other Nail Disease': 10,
 'Poison Ivy Photos and other Contact Dermatitis': 11,
 'Psoriasis pictures Lichen Planus and related diseases': 12,
 'Scabies Lyme Disease and other Infestations and Bites': 13,
 'Systemic Disease': 14,
 'Tinea Ringworm Candidiasis and other Fungal Infections': 15,
 'Urticaria Hives': 16,
 'Vascular Tumors': 17,
 'Vasculitis Photos': 18,
 'Warts Molluscum and other Viral Infections': 19,
 'akiec': 20,
 'bcc': 21,
 'bkl': 22,
 'df': 23,
 'healthy': 24,
 'mel': 25,
 'nv': 26,
 'vasc': 27}

In [13]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/device:GPU:0']


In [15]:
from keras.callbacks import EarlyStopping


r = model.fit_generator(
        training_set,
        validation_data=test_set,
        epochs=10,
        steps_per_epoch=len(training_set),
        validation_steps=len(test_set),
        
    )


C:\Users\RAHUL NAIR\AppData\Local\Temp\ipykernel_2095508\1022773569.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
2850/2850 [==============================] - 802s 277ms/step - loss: 24.3590 - accuracy: 0.3466 - val_loss: 28.1888 - val_accuracy: 0.3735
Epoch 2/10
2850/2850 [==============================] - 739s 259ms/step - loss: 23.3716 - accuracy: 0.4561 - val_loss: 26.8178 - val_accuracy: 0.4483
Epoch 3/10
2850/2850 [==============================] - 709s 249ms/step - loss: 22.1700 - accuracy: 0.5068 - val_loss: 30.3353 - val_accuracy: 0.4631
Epoch 4/10
2850/2850 [==============================] - 809s 284ms/step - loss: 21.2649 - accuracy: 0.5416 - val_loss: 30.0530 - val_accuracy: 0.4733
Epoch 5/10
2850/2850 [==============================] - 866s 304ms/step - loss: 20.7435 - accuracy: 0.5660 - val_loss: 31.0009 - val_accuracy: 0.4847
Epoch 6/10
2850/2850 [==============================] - 869s 305ms/step - loss: 19.9078 - accuracy: 0.5898 - val_loss: 30.4672 - val_accuracy: 0.5045
Epoch 7/10
2850/2850 [==============================] - 692s 243ms/step - loss: 19.3098 - accuracy: 

In [16]:
model.evaluate(testing_set)

1056/1056 [==============================] - 529s 501ms/step - loss: 34.9740 - accuracy: 0.5091


[34.97400665283203, 0.5090882182121277]

In [17]:
import tensorflow as tf
from keras.models import load_model
tf.saved_model.save(model,"../models/model_v6_resnet_tf_augmented")
#model.save('../models/model_v5_resnetl.h5')

INFO:tensorflow:Assets written to: ../models/model_v6_resnet_tf_augmented\assets


INFO:tensorflow:Assets written to: ../models/model_v6_resnet_tf_augmented\assets


In [ ]:
saved_model_dir="../models/model_v6_resnet_tf_augmented"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open('../models/tflite/modelResnetaugmentedV2_all.tflite', 'wb') as f:
  f.write(tflite_model)